In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import sys; sys.path.insert(0, '../') 
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from scipy.spatial.distance import cdist
import mne

from invert.forward import get_info, create_forward_model
from invert.util import pos_from_forward
from invert.evaluate import eval_mean_localization_error

pp = dict(surface='inflated', hemi='both', verbose=0, cortex='low_contrast')

In [2]:
import mne
import pickle as pkl
modality = "MEG"
fwd = mne.read_forward_solution(f"{modality}_Clean_low-fwd.fif")
fwd = mne.convert_forward_solution(fwd, force_fixed=True)

with open(f"{modality}_info.pkl", "rb") as f:
    info = pkl.load(f)

pos = pos_from_forward(fwd)
leadfield = fwd["sol"]["data"]
n_chans, n_dipoles = leadfield.shape

source_model = fwd['src']
vertices = [source_model[0]['vertno'], source_model[1]['vertno']]
adjacency = mne.spatial_src_adjacency(fwd["src"], verbose=0)
distance_matrix = cdist(pos, pos)
fwd

Reading forward solution from c:\Users\lukas\OneDrive\Dokumente\projects\invert\dev\MEG_Clean_low-fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (1284 sources, 305 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


Good channels,"203 Gradiometers, 102 Magnetometers"
Bad channels,None
Source space,Surface with 1284 vertices
Source orientation,Fixed


# Simulate

In [82]:
from invert.simulate import generator
# signal = np.random.randn(n_dipoles, 200)
# signal_Y = leadfield @ signal
# noise_cov = signal_Y @ signal_Y.T + np.identity(n_chans) * 1e-10

# plt.figure()
# plt.imshow(noise_cov)
# plt.colorbar()
# channel_types = fwd["info"].get_channel_types()
# for ch_type in set(channel_types):
#     selection = np.where(np.array(channel_types) == ch_type)[0]
#     plt.figure()
#     plt.imshow(noise_cov[selection][:, selection])
#     plt.colorbar()
#     plt.title(ch_type)

sim_params = dict(
    use_cov=False,
    return_mask=False,
    batch_repetitions=1,
    batch_size=1,
    n_sources=2,
    n_orders=(0, 0),
    snr_range=(0, 0),
    amplitude_range=(1, 1),
    n_timecourses=200,
    n_timepoints=200,
    scale_data=False,
    add_forward_error=False,
    forward_error=0.1,
    inter_source_correlation=0.5,
    return_info=True,
    diffusion_parameter=0.1,
    beta_range = (1, 1),  # Determines the frequency spectrum of each simulted time course (1/f**beta)
    
    correlation_mode=None,
    noise_color_coeff=0.,

    # correlation_mode="cholesky",
    # noise_color_coeff=0.5,

    # correlation_mode = noise_cov,
    # noise_color_coeff="low",
    
    normalize_leadfield=False,
    random_seed=None)

gen = generator(fwd, **sim_params)

In [65]:
X_noise = np.random.randn(305, 200)
cov = np.linalg.cholesky(noise_cov)
X_noise_full = np.zeros_like(X_noise)
channel_types = fwd["info"].get_channel_types()
for ch_type in set(channel_types):
    selection = np.where(np.array(channel_types) == ch_type)[0]
    # X_noise_full[selection] = cov[selection][:, selection] @ X_noise[selection]
    X_noise_full[selection] = np.linalg.cholesky(noise_cov[selection][:, selection]) @ X_noise[selection]
    plt.figure()
    plt.subplot(311)
    plt.imshow(noise_cov[selection][:, selection])
    plt.colorbar()
    plt.title(f"Noise cov at {ch_type}")

    plt.subplot(312)
    C_real = X_noise_full[selection] @ X_noise_full[selection].T
    plt.imshow(C_real)
    plt.colorbar()
    plt.title(f"Effective Noise cov at {ch_type}")

    plt.subplot(313)
    plt.plot(X_noise_full[selection].T)
    


In [83]:
x, y, sim_info = gen.__next__()

tmin = 0
tstep = 1/info["sfreq"]
subject = "fsaverage"
evoked = mne.EvokedArray(x[0].T, info, tmin=tmin)
stc = mne.SourceEstimate(y[0].T, vertices, tmin=tmin, tstep=tstep, 
                        subject=subject, verbose=0)

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, "Ground Truth", 'title',
               font_size=14)
evoked.plot_joint(title="Ground Truth")

evoked_clean = mne.EvokedArray(leadfield @ y[0].T, info, tmin=tmin)
evoked_clean.plot_joint(title="Ground Truth Noiseless")

evoked_noise = evoked.copy()
evoked_noise.data -= evoked_clean.data
evoked_noise.plot_joint(title="Ground Truth Noise")

203 grad
Effective SNR for grad is 1.0014303030506475
102 mag
Effective SNR for mag is 1.0092936120875982
effectice snr overall is 1.0014410684870152
Created an SSP operator (subspace dimension = 3)
3 projection items activated
SSP projectors applied...
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
Created an SSP operator (subspace dimension = 3)
3 projection items activated
SSP projectors applied...
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
Created an SSP operator (subspace dimension = 3)
3 projection items activated
SSP projectors applied...
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <

[<Figure size 800x420 with 6 Axes>, <Figure size 800x420 with 6 Axes>]

In [10]:
a = "b"
b = np.random.rand(1,10)
type(a) == str
type(b) == np.ndarray

True

# Solve

In [25]:
from invert import Solver
prep_leadfield = False
fwd_ = fwd.copy()
fwd_["sol"]["data"] -= fwd_["sol"]["data"].mean(axis=0)
prep_leadfield = True
evoked.set_eeg_reference(verbose=0).apply_proj()
solver = Solver("SSM", prep_leadfield=prep_leadfield)
solver.make_inverse_operator(fwd, evoked, 
                                alpha="auto", n=2, k=2, 
                                n_orders=0, max_iter=5)

stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), 
                            adjacency.toarray(), distance_matrix, mode="match")
print("Standard", mle)

brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, "SSM Standard", 'title',
               font_size=14)

solver = Solver("SSM", prep_leadfield=prep_leadfield)
solver.make_inverse_operator(fwd_, evoked, 
                                alpha="auto", n=2, k=2, 
                                n_orders=0, max_iter=5)

stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), 
                            adjacency.toarray(), distance_matrix, mode="match")
print("CAR", mle)

brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, "SSM CAR", 'title',
               font_size=14)

# 
solver = Solver("SSM", prep_leadfield=prep_leadfield)
solver.make_inverse_operator(fwd_, evoked, 
                                alpha="auto", n=2, k=2, 
                                n_orders=0, max_iter=5,
                                lambda_reg1=0.001*0.2,
                                lambda_reg2=0.0001*0.2)

stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), 
                            adjacency.toarray(), distance_matrix, mode="match")
print("CAR", mle)

brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, "SSM CAR", 'title',
               font_size=14)

No projector specified for this dataset. Please consider the method self.add_proj.
Standard 52.55289661687765


C:\Users\lukas\AppData\Local\Temp\ipykernel_9820\277064062.py:20: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


CAR 30.284960246280665


C:\Users\lukas\AppData\Local\Temp\ipykernel_9820\277064062.py:37: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


CAR 30.284960246280665


C:\Users\lukas\AppData\Local\Temp\ipykernel_9820\277064062.py:57: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


In [28]:
solver = Solver("SSM", prep_leadfield=prep_leadfield)
solver.make_inverse_operator(fwd_, evoked, 
                                alpha="auto", n=2, k=2, 
                                n_orders=0, max_iter=5,
                                lambda_reg1=0.001*100,
                                lambda_reg2=0.0001*100)

stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), 
                            adjacency.toarray(), distance_matrix, mode="match")
print("CAR", mle)

brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, "SSM CAR", 'title',
               font_size=14)

CAR 8.276178393572605


C:\Users\lukas\AppData\Local\Temp\ipykernel_9820\3786372506.py:16: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


# SSM with and without

In [75]:
from invert import Solver
from tqdm.notebook import tqdm
from copy import deepcopy

errors = []

n_samples = 200
tmin = 0
tstep = 1/info["sfreq"]
subject = "fsaverage"
sim_params["random_seed"] = 40
sim_params["normalize_leadfield"] = False
sim_params["inter_source_correlation"] = 0.5
sim_params["snr_range"] = (0, 0)
sim_params["n_timepoints"] = 200
lambda_reg1 = 0.001
lambda_reg2 = 0.0001
factors = (0.1, 59/306, 1, 306/59, 10, 100,)
params = {
    # "standard": dict(lambda_reg1=0.001, lambda_reg2=0.0001),
    "new_standard": dict(lambda_reg1=0.001*100, lambda_reg2=0.0001*1),
    }
# for factor1 in factors:
#     for factor2 in factors:
#         params[f"factor_{factor1}_{factor2}"] = dict(lambda_reg1=lambda_reg1*factor1, lambda_reg2=lambda_reg2*factor2)

solver_names = ["SSM", "AP"]
# solver_names = ["SSM",]
for isc in (0.5, 0.95):
    sim_params["inter_source_correlation"] = isc
    for n_timepoints in (50, 1000):
        sim_params["n_timepoints"] = n_timepoints
        for normalize_leadfield in (False, True):  # True):
            sim_params["normalize_leadfield"] = normalize_leadfield
            gen = generator(fwd, **sim_params)
            for i_sample in tqdm(range(n_samples)):
                x, y, sim_info = gen.__next__()
                print(f"Sample {i_sample}/{n_samples}")
                for kind, param in params.items():
                    for solver_name in solver_names:
                        print(f"\t{solver_name}")
                        for normalize_solver in (False, ):  # True):
                            for car_leadfield in (False,):  # False):
                                fwd_ = deepcopy(fwd)
                                if car_leadfield:
                                    fwd_["sol"]["data"] -= fwd_["sol"]["data"].mean(axis=0)
                                evoked = mne.EvokedArray(x[0].T, info, tmin=tmin, verbose=0)
                                if modality == "EEG":
                                    evoked.set_eeg_reference(verbose=0).apply_proj(verbose=0)
                                stc = mne.SourceEstimate(y[0].T, vertices, tmin=tmin, tstep=tstep, 
                                                        subject=subject, verbose=0)

                                solver = Solver(solver_name, prep_leadfield=normalize_solver)
                                solver.make_inverse_operator(fwd_, evoked, alpha="auto", n=2, k=2, n_orders=0, max_iter=6, **param)
                                
                                stc_ = solver.apply_inverse_operator(evoked)

                                stc_.data /= abs(stc_.data).max()
                                
                                mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), 
                                                            adjacency.toarray(), distance_matrix, mode="match")
                                errors.append(dict(solver=solver_name, car_leadfield=car_leadfield, i_sample=i_sample, mle=mle, normalize_leadfield=normalize_leadfield, normalize_solver=normalize_solver, param_kind=kind, isc=isc, n_timepoints=n_timepoints))

                                print("\t\t\tMLE=", round(mle,1))

                            # error_true = errors[-1]["mle"]
                            # print(f"""\n\tError: {error_true:.1f}""")
                            # error_false = errors[-2]["mle"]
                            # s = "Same" if error_true == error_false else "Different"
                            # print(f"""\n\tTrue: {error_true:.1f}, False: {error_false:.1f}\n ({s})""")

  0%|          | 0/200 [00:00<?, ?it/s]

Sample 0/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 1/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 2/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 3/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 4/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 5/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 6/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 7/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 8/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 9/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 10/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 11/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 12/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 13/200
	SSM
			MLE= 22.2
	AP
			MLE= 22.2
Sample 14/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 15/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 16/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 17/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 18/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 19/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 20/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 21/200
	SSM
		

  0%|          | 0/200 [00:00<?, ?it/s]

Sample 0/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 1/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 2/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 3/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 4/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 5/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 6/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 7/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 8/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 9/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 10/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 11/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 12/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 13/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 14/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 15/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 16/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 17/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 18/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 19/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 20/200
	SSM
			MLE= 5.9
	AP
			MLE= 5.9
Sample 21/200
	SSM
			M

  0%|          | 0/200 [00:00<?, ?it/s]

...

Sample 0/200
	SSM
			MLE= 13.3
	AP
			MLE= 13.0
Sample 1/200
	SSM
			MLE= 8.7
	AP
			MLE= 0.0
Sample 2/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 3/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 4/200
	SSM
			MLE= 15.9
	AP
			MLE= 15.9
Sample 5/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 6/200
	SSM
			MLE= 4.9
	AP
			MLE= 0.0
Sample 7/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 8/200
	SSM
			MLE= 7.2
	AP
			MLE= 0.0
Sample 9/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 10/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 11/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 12/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 13/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 14/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 15/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 16/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 17/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 18/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 19/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 20/200
	SSM
			MLE= 13.0
	AP
			MLE= 13.0
Sample 21/200
	SS

  0%|          | 0/200 [00:00<?, ?it/s]

Sample 0/200
	SSM
			MLE= 22.1
	AP
			MLE= 22.1
Sample 1/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 2/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 3/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 4/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 5/200
	SSM
			MLE= 11.6
	AP
			MLE= 11.6
Sample 6/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 7/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 8/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 9/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 10/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 11/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 12/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 13/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 14/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 15/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 16/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 17/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 18/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 19/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 20/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 21/200
	SSM


  0%|          | 0/200 [00:00<?, ?it/s]

Sample 0/200
	SSM
			MLE= 13.3
	AP
			MLE= 13.3
Sample 1/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 2/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 3/200
	SSM
			MLE= 4.3
	AP
			MLE= 0.0
Sample 4/200
	SSM
			MLE= 2.9
	AP
			MLE= 0.0
Sample 5/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 6/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 7/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 8/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 9/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 10/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 11/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 12/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 13/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 14/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 15/200
	SSM
			MLE= 7.3
	AP
			MLE= 0.0
Sample 16/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 17/200
	SSM
			MLE= 7.1
	AP
			MLE= 0.0
Sample 18/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 19/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 20/200
	SSM
			MLE= 0.0
	AP
			MLE= 0.0
Sample 21/200
	SSM
		

In [76]:
import pandas as pd
df = pd.DataFrame(errors)

# import matplotlib.pyplot as plt
# plt.figure()
# plt.scatter(
#     df[df.prep_leadfield == True]["mle"].values,
#     df[df.prep_leadfield == False]["mle"].values,
# )

In [57]:
df.head()

,solver,car_leadfield,i_sample,mle,normalize_leadfield,normalize_solver,param_kind,isc,n_timepoints
0,SSM,True,0,13.436861,False,False,new_standard,0.5,50
1,SSM,True,0,13.436861,False,True,new_standard,0.5,50
2,AP,True,0,13.436861,False,False,new_standard,0.5,50
3,AP,True,0,13.436861,False,True,new_standard,0.5,50
4,SSM,True,1,0.000000,False,False,new_standard,0.5,50


In [77]:
df.groupby(["solver", "isc", "n_timepoints", "normalize_leadfield"]).describe()["mle"]#.sort_values("mean")

count      mean        std  min  \
solver isc  n_timepoints normalize_leadfield                                    
AP     0.50 50           False                200.0  0.625217   4.410828  0.0   
                         True                 200.0  0.093664   0.997715  0.0   
            1000         False                200.0  0.000000   0.000000  0.0   
                         True                 200.0  0.000000   0.000000  0.0   
       0.95 50           False                200.0  1.923827   6.578658  0.0   
                         True                 200.0  3.790915   9.879204  0.0   
            1000         False                200.0  2.066880   6.205769  0.0   
                         True                 200.0  3.581009   9.518188  0.0   
SSM    0.50 50           False                200.0  0.490779   3.768954  0.0   
                         True                 200.0  0.093664   0.997715  0.0   
            1000         False                200.0  0.000000   0.000000  0.0   
                         True                 200.0  0.000000   0.000000  0.0   
       0.95 50           False                200.0  1.969453   6.512816  0.0   
                         True                 200.0  4.559497  10.051612  0.0   
            1000         False                200.0  2.234858   6.259898  0.0   
                         True                 200.0  4.310614  10.105959  0.0   

                                              25%  50%       75%        max  
solver isc  n_timepoints normalize_leadfield                                 
AP     0.50 50           False                0.0  0.0  0.000000  46.127162  
                         True                 0.0  0.0  0.000000  12.858408  
            1000         False                0.0  0.0  0.000000   0.000000  
                         True                 0.0  0.0  0.000000   0.000000  
       0.95 50           False                0.0  0.0  0.000000  46.757370  
                         True                 0.0  0.0  0.000000  61.341884  
            1000         False                0.0  0.0  0.000000  33.030947  
                         True                 0.0  0.0  0.000000  61.376441  
SSM    0.50 50           False                0.0  0.0  0.000000  46.127162  
                         True                 0.0  0.0  0.000000  12.858408  
            1000         False                0.0  0.0  0.000000   0.000000  
                         True                 0.0  0.0  0.000000   0.000000  
       0.95 50           False                0.0  0.0  0.000000  46.757370  
                         True                 0.0  0.0  3.124498  61.341884  
            1000         False                0.0  0.0  0.000000  33.030947  
                         True                 0.0  0.0  0.000000  61.376441

In [35]:
df.groupby(["solver", "isc"]).describe()["mle"]

count       mean        std  min  25%        50%        75%  \
solver isc                                                                 
AP     0.50   50.0   1.204060   5.960306  0.0  0.0   0.000000   0.000000   
       0.95   50.0  19.948270  19.136229  0.0  0.0  21.920248  35.452912   
SSM    0.50   50.0   1.950509   6.972664  0.0  0.0   0.000000   0.000000   
       0.95   50.0  17.345430  21.054543  0.0  0.0  10.054456  30.333040   

                   max  
solver isc              
AP     0.50  30.817502  
       0.95  57.178449  
SSM    0.50  37.417607  
       0.95  62.535069

In [22]:
df.iloc[40:].groupby(["solver", ]).describe()["mle"]

,count,mean,std,min,25%,50%,75%,max
solver,,,,,,,,
AP,20.0,19.280181,17.168929,0.0,6.477809,16.223874,30.670395,66.062789
SSM,20.0,26.743466,20.586806,0.0,8.506317,24.926203,45.797094,58.099114


In [28]:
import seaborn as sns
plt.figure()
for i, normalize_leadfield in enumerate((True, )):
    df_ = df.copy()
    df_[(df_.normalize_leadfield==normalize_leadfield) & (df_.normalize_solver==True)]
    plt.subplot(211+i)
    sns.barplot(data=df_, x="isc", hue="solver", y="mle")
    title = "Normalized Leadfield" if normalize_leadfield else "Original Leadfield"
    plt.title(title)
    if i == 1:
        plt.xlabel("Centered Leadfield Columns")
        plt.ylabel("Mean Localization Error (mm)")
    else:
        plt.xlabel("")
        plt.ylabel("")
        

In [121]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure()
sns.barplot(x="solver", hue="car_leadfield", y="mle", data=df, estimator=np.mean)
plt.show()

In [12]:
df.groupby("solver").describe()
import seaborn as sns
from matplotlib import pyplot as plt
plt.figure()
sns.barplot(data=df, x="normalize_leadfield", hue="solver", y="mle", estimator="mean")

<Axes: xlabel='normalize_leadfield', ylabel='mle'>

In [ ]:
df.groupby("prep_leadfield").describe()

In [ ]:
df.groupby("prep_leadfield").describe()

# Leadfield comparison

In [82]:
fwd["sol"]["data"].std()**2

1084453848.2834473

In [91]:
import mne
import pickle as pkl
# modality = "EEG"
variances = dict()
for modality in ("EEG", "MEG"):
    fwd = mne.read_forward_solution(f"{modality}_Clean_coarse-fwd.fif")
    fwd = mne.convert_forward_solution(fwd, force_fixed=True)
    # fwd["sol"]["data"] /= abs(fwd["sol"]["data"]).mean()
    norms = np.linalg.norm(fwd["sol"]["data"], axis=0)
    print(modality, norms.max()/norms.min())
    variances[modality] = fwd["sol"]["data"].std(axis=0)**2

plt.figure()
plt.scatter(variances["EEG"], variances["MEG"])
# linear regression line
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(variances["EEG"], variances["MEG"])
plt.plot(variances["EEG"], slope*variances["EEG"] + intercept, color="red")
# plot this mean and std cross within the scatter plot
# plt.scatter(variances["EEG"].mean(), variances["MEG"].mean(), color="red", s=100)
plt.errorbar(variances["EEG"].mean(), variances["MEG"].mean(), xerr=variances["EEG"].std(), yerr=variances["MEG"].std(), fmt="none", color="black")
plt.xlabel("Variances EEG")
plt.ylabel("Variances MEG")
plt.title("Leadfield Column Variances")


Reading forward solution from c:\Users\lukas\OneDrive\Dokumente\projects\invert\dev\EEG_Clean_coarse-fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read EEG forward solution (20484 sources, 59 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
EEG 13.247724
Reading forward solution from c:\Users\lukas\OneDrive\Dokumente\projects\invert\dev\MEG_Clean_coarse-fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (20484 sources, 305 channe

Text(0.5, 1.0, 'Leadfield Column Variances')

# RAP-NN

In [ ]:
# from invert.solvers.esinet import SolverRAPNN

# solver_rapn = SolverRAPNN()

# solver_rapn.make_inverse_operator(fwd, evoked, n_orders=(0, 0), 
#                                   batch_size=1284, batch_repetitions=5,
#                                   n_dense_units=100, n_dense_layers=1,
#                                   snr_range=(0.2, 100), n_timepoints=20, 
#                                   inter_source_correlation=(0,1),
#                                   epochs=1,
#                                   correlation_mode="cholesky",
#                                   noise_color_coeff=(0, 0.5),
#                                   n_sources=(1, 5), verbose=1)
# # solver_rapn.train_model()
solver_rapn.model.load_weights('.weights.h5', skip_mismatch=True)

In [ ]:
from invert.simulate import generator
sim_params = dict(
    use_cov=False,
    return_mask=False,
    batch_repetitions=1,
    batch_size=1,
    n_sources=3,
    n_orders=(0, 0),
    snr_range=(5, 5),
    amplitude_range=(1, 1),
    n_timecourses=200,
    n_timepoints=20,
    scale_data=False,
    add_forward_error=False,
    forward_error=0.1,
    inter_source_correlation=0.5,
    return_info=True,
    diffusion_parameter=0.1,
    correlation_mode="cholesky",
    noise_color_coeff=0.5,
    # correlation_mode=None,
    # noise_color_coeff=0.0,
    random_seed=None)

gen = generator(fwd, **sim_params)

x, y, sim_info = gen.__next__()

tmin = 0
tstep = 1/info["sfreq"]
subject = "fsaverage"
evoked = mne.EvokedArray(x[0].T, info, tmin=tmin)
stc = mne.SourceEstimate(y[0].T, vertices, tmin=tmin, tstep=tstep, 
                        subject=subject, verbose=0)

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, "Ground Truth", 'title',
               font_size=14)
evoked.plot_joint(title="Ground Truth")

# evoked_clean = mne.EvokedArray(leadfield @ y[0].T, info, tmin=tmin)
# evoked_clean.plot_joint(title="Ground Truth Noiseless")

# evoked_noise = evoked.copy()
# evoked_noise.data -= evoked_clean.data
# evoked_noise.plot_joint(title="Ground Truth Noise")

In [ ]:
stc_ = solver_rapn.apply_inverse_operator(evoked, sim_params["n_sources"], 
                                            refine_solution=False, 
                                            alpha=1, max_iter=100)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver_rapn.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(
    fwd["sol"]["data"] @ stc_.data, info
    ).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), 
                                   adjacency.toarray(), distance_matrix, mode="match")
print(solver_rapn.name, "mle = ", mle, " mm")

In [ ]:
for name, estimate in solver_rapn.estimates.items():
    stc_.data = estimate[:, np.newaxis]
    brain = stc_.plot(**pp)
    brain.add_text(0.1, 0.9, name, 'title',
                     font_size=14)

## SSM

In [ ]:
from invert import Solver
n_sources = sim_params["n_sources"]
solver = Solver("ap")
solver.make_inverse_operator(fwd, evoked, 
                             n_orders=sim_params["n_orders"][1], 
                             refine_solution=True, n=n_sources, 
                             k=n_sources, diffusion_parameter=0.1, 
                             stop_crit=0, max_iter=6)

stc_ = solver.apply_inverse_operator(evoked)
# stc_.data /= abs(stc_.data).max()
# brain = stc_.plot(**pp)
# brain.add_text(0.1, 0.9, solver.name, 'title',
#                font_size=14)

# evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
# evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix, mode="match")
print(solver.name, "mle = ", mle, " mm")

# Eval many

In [ ]:
from invert.simulate import generator
sim_params = dict(
    use_cov=False,
    return_mask=False,
    batch_repetitions=1,
    batch_size=1,
    n_sources=2,
    n_orders=(0, 0),
    snr_range=(1, 1),
    amplitude_range=(1, 1),
    n_timecourses=200,
    n_timepoints=20,
    scale_data=False,
    add_forward_error=False,
    forward_error=0.1,
    inter_source_correlation=0.9,
    return_info=True,
    diffusion_parameter=0.1,
    correlation_mode="cholesky",
    noise_color_coeff=0.5,
    # correlation_mode=None,
    # noise_color_coeff=0.0,
    random_seed=None)

gen = generator(fwd, **sim_params)

solver = Solver("ap")
solver.verbose=0
solver_rapn.verbose=0
results = []
for i_samp in range(100):
    print(i_samp)
    x, y, sim_info = gen.__next__()
    evoked = mne.EvokedArray(x[0].T, info, tmin=tmin)
    stc = mne.SourceEstimate(y[0].T, vertices, tmin=tmin, tstep=tstep,
                                subject=subject, verbose=0)

    stc_ = solver_rapn.apply_inverse_operator(evoked, sim_params["n_sources"], 
        refine_solution=True, 
        alpha=1, max_iter=10)
    

    mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix, mode="match")
    results.append(dict(solver="CovCNN", mle=mle))

    solver.make_inverse_operator(fwd, evoked, 
                             n_orders=sim_params["n_orders"][1], 
                             refine_solution=False, n=n_sources, 
                             k=n_sources, diffusion_parameter=0.1, 
                             stop_crit=0, max_iter=6)

    stc_ = solver.apply_inverse_operator(evoked)

    mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix, mode="match")
    results.append(dict(solver="AP", mle=mle))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(results)
plt.figure()
# sns.boxplot(data=df, x="method", y="MLE")
sns.barplot(data=df, x="solver", y="mle", estimator=np.mean)
title = f"""sources: {sim_params["n_sources"]}, snr: {sim_params["snr_range"][0]}, isc: {sim_params["inter_source_correlation"]}"""
plt.title(title)
plt.ylim(0, 40)

df.groupby("solver").describe()["mle"]

## CovCNN

In [ ]:
from invert.solvers.esinet import SolverCovCNN, SolverCovLSTM2

from copy import deepcopy

train_params = deepcopy(sim_params)
train_params["batch_repetitions"] = 10
train_params["batch_size"] = 1284
train_params["n_filters"] = 128
train_params["n_lstm_units"] = 32
train_params["n_dense_units"] = 300
train_params["epochs"] = 50
train_params["n_orders"] = (0, 1)
train_params["snr_range"] = [0.1, 10]
train_params["n_sources"] = (1, 5)
train_params["inter_source_correlation"] = (0, 1)
train_params["output_activation"] = "sigmoid"
train_params["loss"] = "cosine_similarity"
train_params["verbose"] = 1


# solver_covlstm = SolverCovLSTM2()
# solver_covlstm.make_inverse_operator(fwd, evoked, **train_params)
## solver_covlstm.train_model()

# print()
# print()

# solver_covcnn = SolverCovCNN()
# solver_covcnn.make_inverse_operator(fwd, evoked, **train_params)
solver_covcnn.train_model()

In [ ]:
from invert.evaluate import emd as get_emd

stc_ = solver_covcnn.apply_inverse_operator(evoked)
evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)

# stc_.data /= abs(stc_.data).max()
# brain = stc_.plot(**pp)
# brain.add_text(0.1, 0.9, solver_covcnn.name, 'title',
#                font_size=14)

stc_.data = solver_covcnn.gammas[:, np.newaxis]
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, "Gamma", 'title',
               font_size=14)

evoked_.plot_joint()

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix, mode="match")
emd = get_emd(pos, stc.data[:, 0], pos, stc_.data[:, 0])
print(solver_covcnn.name, "mle = ", mle, " mm", "emd = ", emd, " mm")


# stc_ = solver_covlstm.apply_inverse_operator(evoked)
# evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)

# # stc_.data /= abs(stc_.data).max()
# # brain = stc_.plot(**pp)
# # brain.add_text(0.1, 0.9, solver_covlstm.name, 'title',
# #                font_size=14)

# stc_.data = solver_covlstm.gammas[:, np.newaxis]
# brain = stc_.plot(**pp)
# brain.add_text(0.1, 0.9, "Gamma", 'title',
#                font_size=14)

# evoked_.plot_joint()

# mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix, mode="match")
# emd = get_emd(pos, stc.data[:, 0], pos, stc_.data[:, 0])
# print(solver_covcnn.name, "mle = ", mle, " mm", "emd = ", emd, " mm")


In [ ]:
evoked.data.shape

## LCMV

In [ ]:
from invert import Solver
solver = Solver("dSPM", prep_leadfield=False, n_reg_params=125)
solver.make_inverse_operator(fwd, evoked)

stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")

In [ ]:
from invert import Solver
solver = Solver("dSPM", prep_leadfield=True)
solver.make_inverse_operator(fwd, evoked)

stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")

# S-FLEX

In [ ]:
import numpy as np
from scipy.optimize import minimize

import numpy as np

def nip_translf(L):
    """
    Transforms a Leadfield matrix into a tensor or vice versa.
    Each slice of the tensor corresponds to the lead field matrix in one direction (x, y, or z).
    """
    if L.ndim == 3:
        # From tensor to matrix
        Ltrans = np.reshape(np.transpose(L, (0, 2, 1)), (L.shape[0], L.shape[1] * 3))
    elif L.ndim == 2:
        # From matrix to tensor
        if L.shape[1] % 3 != 0:
            raise ValueError("The second dimension of L must be divisible by 3")
        Ltrans = np.zeros((L.shape[0], L.shape[1] // 3, 3))
        for i in range(3):
            Ltrans[:, :, i] = L[:, np.arange(i, L.shape[1], 3)]
    else:
        raise ValueError("L should have at least 2 dimensions")
    return Ltrans


def nip_sflex(y, L, basis, maxiter=50, resnorm=0.5, regpar=1e-2, optimres=False, Winv=None):
    Nc, Nt = y.shape
    Nd, _ = L.shape
    L = nip_translf(L)
    nbasis = basis.shape[1]
    NDUM = 3

    xx0f = np.zeros((3 * Nt, nbasis)).flatten()  # Flatten the initial guess for optimization

    def objective(xx):
        # Example objective function; replace with your actual function
        return np.sum((xx - xx0f)**2)  # Placeholder

    def constraint(xx):
        # Example constraint; replace with your actual constraint
        return np.linalg.norm(xx) - 1  # Placeholder

    cons = [{'type': 'eq', 'fun': constraint}]

    options = {'maxiter': 1000, 'disp': True}
    result = minimize(objective, xx0f, method='L-BFGS-B', options=options, constraints=cons)
    xx = result.x

    # Reshape xx back to the original shape if needed
    xx = xx.reshape((3 * Nt, nbasis))  # Adjust the shape based on your specific needs

    # Process xx as needed to compute J_rec and other outputs

    return xx, result


def xforth(x):
    Q = x.shape[1]
    in1, _ = np.nonzero(x)
    in2 = np.unique(np.ceil(in1 / (Nt)).astype(int))
    l = len(in2) // NDUM
    if l == 0:
        xfo = np.zeros((Nc * Nt, Q))
    else:
        xfo = (L[:, in2] @ x[in1, :].reshape(NDUM, Nt, l, Q).transpose(0, 2, 1, 3).reshape(NDUM * l, Nt * Q)).reshape(Nc * Nt, Q)
    return xfo

def xback(x):
    return (L.T @ x.reshape(Nc, Nt)).reshape(NDUM, nbasis, Nt).transpose(0, 2, 1).ravel()

# Example usage:
# Define y, L, and basis matrices with appropriate dimensions and values.
# y = np.random.randn(Nc, Nt)
# L = np.random.randn(Nc, Nd)
# basis = np.random.randn(Nd, nbasis)
# J_rec, extras = nip_sflex(y, L, basis)
L = fwd["sol"]["data"]
y = evoked.data
basis = np.eye(n_dipoles)
J_rec, extras = nip_sflex(y[:, 0][:, np.newaxis], L, basis)

## FC

In [ ]:
from invert import Solver
from copy import deepcopy

solver = Solver("FC")
# solver.make_inverse_operator(fwd, evoked, n_sources=2, n_orders=(0,1),snr_range=(0.2,0.2))
sim_params_train = deepcopy(sim_params)
sim_params_train["batch_repetitions"] = 5
sim_params_train["batch_size"] = 1284
sim_params_train["epochs"] = 1
solver.make_inverse_operator(fwd, evoked, **sim_params_train)
stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")

In [ ]:
solver.save("../models")

## FLAP

In [ ]:
from invert.solvers.music import SolverGeneralizedIterative

solver = SolverGeneralizedIterative()
solver = Solver("AP")
solver.make_inverse_operator(fwd, evoked, 
                             inverse_type="AP", n_orders=0, 
                             refine_solution=True, n=sim_params["n_sources"], 
                             d=sim_params["n_sources"], diffusion_parameter="auto", 
                             lambda_reg3=0.001)
stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")

In [ ]:
from invert.solvers.empirical_bayes import SolverNLChampagne
alpha = "auto"
solver = SolverNLChampagne()
solver.make_inverse_operator(fwd, evoked, alpha=alpha)
stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

In [ ]:
from invert import Solver
alpha = "auto"
solver = Solver("convexity champagne", n_reg_params=20)
solver.make_inverse_operator(fwd, evoked, alpha=alpha)
stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

In [ ]:
from invert import Solver
alpha = 0.1
solver = Solver("Gamma-MAP")
solver.make_inverse_operator(fwd, evoked, alpha=alpha)
stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

# Other

In [12]:
# from invert import Solver
# solver = Solver("SSM")
# solver.make_inverse_operator(fwd, evoked, n=2, k=2, max_iter=6)

# stc_ = solver.apply_inverse_operator(evoked)

# stc_.data /= abs(stc_.data).max()
# brain = stc_.plot(**pp)
# brain.add_text(0.1, 0.9, solver.name, 'title',
#                font_size=14)

# evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
# evoked_.plot_joint()

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), 
                                        adjacency.toarray(), distance_matrix, mode="match")
print(f"Mean Localization Error: {mle:.1f} mm")

Mean Localization Error: 16.9 mm


In [ ]:
from invert.ensemble import Ensemble
solver_list = ["wMNE", "sLORETA", "LAURA", "Convexity Champagne", "GAMMA-MAP", "HOCMCMV", "SSMP", "FLEX-MUSIC"]

solver = Ensemble(solver_list, summary_type="mean")
solver.make_inverse_operator(fwd, evoked)

stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

print("r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])
print("L1 = ",  np.linalg.norm(stc_.data, ord=1))

In [ ]:
from invert.solvers import SolverFLEXMUSIC_2
solver = SolverFLEXMUSIC_2()
solver.make_inverse_operator(fwd, evoked, alpha="auto")

stc_ = solver.apply_inverse_operator(evoked)

stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

print("r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])
print("L1 = ",  np.linalg.norm(stc_.data, ord=1))

# Big Loop

In [ ]:
from invert import Solver
# from invert.config import all_solvers
from invert.evaluate import nmse, corr, eval_mean_localization_error, eval_auc
from invert.adapters import contextualize_bd
import pickle as pkl


all_solvers = [ "MNE", "wMNE", "dSPM", "FISTA", "L1L2",
                "LORETA", "sLORETA", "eLORETA", 
                "LAURA", "Backus-Gilbert", 
                "S-MAP",
                "Champagne", "Multiple Sparse Priors", "Bayesian LORETA", "Bayesian MNE", "Bayesian Beamformer", "Bayesian Beamformer LORETA",
                "BCS", "Gamma-MAP", "Source-MAP", "Gamma-MAP-MSP", "Source-MAP-MSP",
                "MVAB", "LCMV", "SMV", "WNMV", "HOCMV", "ESMV", "MCMV", "ESMCMV", "ReciPSIICOS", "SAM",
                # "Fully-Connected", 
                # "LUCAS",
                # "OMP", "COSAMP", "SOMP", "REMBO", "SP", "SSP",
                # "SMP", "SSMP", "SubSMP",
                "EPIFOCUS",
                "MUSIC", "RAP-MUSIC", "TRAP-MUSIC", "JAZZ-MUSIC",
            ]
redo_solvers = np.array(["Champagne", "Multiple Sparse Priors", 
            "Bayesian LORETA", "Bayesian MNE", "Bayesian Beamformer", 
            "Bayesian Beamformer LORETA", "Gamma-MAP", "Source-MAP"])

all_solvers = ["TRAP-MUSIC", "JAZZ-MUSIC", "LCMV", "sLORETA"]

settings = dict(
    number_of_sources=(1, 10), 
    extents=(1, 25), 
    duration_of_trial=1, 
    target_snr=(1, 100))
alpha = "auto"

errors_corr = {sname: [] for sname in all_solvers}
errors_mle = {sname: [] for sname in all_solvers}
errors_auc = {sname: [] for sname in all_solvers}
errors_nmse = {sname: [] for sname in all_solvers}

solvers = dict()
sim = Simulation(fwd, info, settings).simulate(100)
    
for i in range(sim.n_samples):
    print(i)
    stc = sim.source_data[i]
    evoked = sim.eeg_data[i].average()

    for solver_name in all_solvers :
        print(solver_name)
        solver = Solver(solver=solver_name)
        if (not solver_name in solvers) or solver_name in redo_solvers:
            if solver_name == "Fully-Connected":
                solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha=alpha, settings=dict(duration_of_trial=0., target_snr=settings["target_snr"], extents=settings["extents"]), n_simulations=5000)
            else:
                solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha=alpha)
        # stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))
        # error = np.mean(corr(stc.data, stc_hat.data))
        
        try:
            stc_hat = solvers[solver_name].apply_inverse_operator(evoked)
            error_corr = np.mean(corr(stc.data, stc_hat.data))

            error_mle = eval_mean_localization_error(stc.data[:, 0], stc_hat.data[:, 0], pos)
            error_auc = eval_auc(stc.data[:, 0], stc_hat.data[:, 0], pos)
            error_nmse = np.median(nmse(stc.data, stc_hat.data))

        except:
            print("\t...didnt work this time")
            error_mle = np.nan
            error_auc = np.nan
            error_corr = np.nan
            error_nmse = np.nan

        errors_corr[solver_name].append( error_corr )
        errors_mle[solver_name].append( error_mle )
        errors_auc[solver_name].append( error_auc )
        errors_nmse[solver_name].append( error_nmse )


    print("Saving errors")
    fn = "errors.pkl"
    with open(fn, 'wb') as f:
        pkl.dump([errors_corr, errors_mle, errors_auc, errors_nmse, sim.simulation_info], f)

In [ ]:
# import pickle as pkl
# fn = "errors.pkl"
# with open(fn, 'rb') as f:
#     errors_corr, errors_mle, errors_auc = pkl.load(f)

In [ ]:
# for key, val in errors_mle.items():
#     errors_mle[key] = val[:31]

# for key, val in errors_corr.items():
#     errors_mle[key] = val[:31]

# for key, val in errors_auc.items():
#     errors_mle[key] = val[:31]
    

# Plot

In [ ]:
import pandas as pd
import seaborn as sns
sns.set(font_scale=0.8)
errors = [errors_corr, errors_mle, errors_auc, errors_nmse]
error_names = ["errors_corr", "errors_mle", "errors_auc", "errors_nmse"]
for error, error_name in zip(errors, error_names):
    df = pd.DataFrame(error)


    sorted_index = df.median().sort_values().index
    df = df[sorted_index]

    plt.figure()
    sns.boxplot(data=df)
    plt.title(error_name)


# df_mean_var = pd.concat([df.mean(), df.std()], axis=1)
# df_mean_var = df_mean_var.rename(columns={0: "Median", 1: "Variance"})
# df_mean_var["MedVar"] = df_mean_var["Median"] / df_mean_var["Variance"]
# df_mean_var["Method"] = df_mean_var.index
# df_mean_var = df_mean_var.sort_values(by="Median")
# display(df_mean_var)

# plt.figure()
# sns.scatterplot(x="Median", y="Variance", hue="Method", size="MedVar", data=df_mean_var)
# plt.xlabel("Median")
# plt.ylabel("Variance")


In [ ]:
settings = dict(number_of_sources=3, extents=(25, 35), duration_of_trial=0.025, target_snr=99999)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)